In [ ]:
!pip install -q torchinfo mlflow optuna thop

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 159.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 103.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.2/779.2 kB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.8 MB/s eta 0:00:00


In [ ]:
import os
import math
import random
from collections import defaultdict, Counter
from typing import List, Optional

import h5py
import numpy as np
import pandas as pd
import pywt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Sampler
from scipy.signal import butter, filtfilt, find_peaks
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, roc_auc_score, f1_score
import mlflow
import mlflow.pytorch
import optuna
from optuna.exceptions import TrialPruned
from dotenv import load_dotenv
import time
from thop import profile, clever_format

load_dotenv()

True

In [ ]:
mlflow.set_tracking_uri("databricks")
mlflow.set_experiment("/Users/oladipoeyiara@gmail.com/han_sph_full_run_reduced")


2026/01/06 19:43:43 INFO mlflow.tracking.fluent: Experiment with name '/Users/oladipoeyiara@gmail.com/han_sph_full_run_reduced' does not exist. Creating a new experiment.


<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/4416535780169939', creation_time=1767728623822, experiment_id='4416535780169939', last_update_time=1767728623822, lifecycle_stage='active', name='/Users/oladipoeyiara@gmail.com/han_sph_full_run_reduced', tags={'mlflow.experiment.sourceName': '/Users/oladipoeyiara@gmail.com/han_sph_full_run_reduced',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'oladipoeyiara@gmail.com',
 'mlflow.ownerId': '1108839756692281'}>

In [ ]:
def set_seed(seed):
    torch.manual_seed(seed)

    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import zipfile

# Define the path to the zip file in Google Drive
zip_file_path = '/content/drive/MyDrive/records.zip'

# Define the extraction path
extract_path = '/content'

# Create the extraction directory if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

# Check if the zip file exists
if os.path.exists(zip_file_path):
    # Extract the zip file
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"Successfully extracted {zip_file_path} to {extract_path}")
else:
    print(f"Error: {zip_file_path} not found.")

Mounted at /content/drive
Successfully extracted /content/drive/MyDrive/records.zip to /content


In [ ]:
# -------------------------
# Config
# -------------------------
PATH = "records"
SAMPLING_RATE = 500
PRE_PEAK_SAMPLES = 99
POST_PEAK_SAMPLES = 201
SEGMENT_LENGTH = PRE_PEAK_SAMPLES + POST_PEAK_SAMPLES  # 300
BATCH_SIZE = 16
DROP_LAST = True
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# -------------------------
# utilities: filtering and peak detection
# -------------------------
def denoise(data):
    # wavelet transform
    coeffs = pywt.wavedec(data=data, wavelet='db5', level=9)
    cA9, cD9, cD8, cD7, cD6, cD5, cD4, cD3, cD2, cD1 = coeffs

    # Threshold denoising
    threshold = (np.median(np.abs(cD1)) / 0.6745) * (np.sqrt(2 * np.log(len(cD1))))
    cD1.fill(0)
    cD2.fill(0)
    for i in range(1, len(coeffs) - 2):
        coeffs[i] = pywt.threshold(coeffs[i], threshold)

    # Inverse wavelet transform to obtain the denoised signal
    rdata = pywt.waverec(coeffs=coeffs, wavelet='db5')
    return rdata


def pan_tompkins_detector(ecg_signal, fs):
    lowcut, highcut = 5.0, 15.0
    nyquist = 0.5 * fs
    low, high = lowcut / nyquist, highcut / nyquist
    b, a = butter(1, [low, high], btype='band')
    filtered_ecg = filtfilt(b, a, ecg_signal)
    diff_ecg = np.diff(filtered_ecg)
    squared_ecg = diff_ecg ** 2
    window_size = int(0.150 * fs)
    mwa_ecg = np.convolve(squared_ecg, np.ones(window_size) / window_size, mode='same')
    peaks, _ = find_peaks(mwa_ecg, distance=int(0.6 * fs))
    return peaks


def multi_lead_fusion(detected_peaks, fs, fusion_window=0.1, min_leads=None):
    n_leads = len(detected_peaks)
    if min_leads is None:
        min_leads = int(np.ceil(n_leads / 2))

    # Collect all peaks with their lead information
    all_peaks = [(p, lead) for lead, peaks in enumerate(detected_peaks) for p in peaks]
    all_peaks.sort(key=lambda x: x[0])

    fused_peaks = []
    i = 0

    while i < len(all_peaks):
        # Start a new cluster
        cluster = [all_peaks[i]]
        i += 1

        # Add nearby peaks to the cluster
        while i < len(all_peaks) and all_peaks[i][0] - cluster[-1][0] <= fusion_window * fs:
            cluster.append(all_peaks[i])
            i += 1

        # Check if cluster has peaks from enough leads
        unique_leads = {lead for _, lead in cluster}
        if len(unique_leads) >= min_leads:
            # Use median position as the fused peak
            fused_peak = int(np.median([p for (p, _) in cluster]))
            fused_peaks.append(fused_peak)

    return np.array(sorted(fused_peaks))


def detect_r_peaks(ecg_signals, fs):
    detected_peaks = []
    for lead in ecg_signals:
        peaks = pan_tompkins_detector(lead, fs)
        detected_peaks.append(peaks)

    fused_r_peaks = multi_lead_fusion(detected_peaks, fs, fusion_window=0.1, min_leads=6)
    return fused_r_peaks


def extract_segments_around_peaks(signal, r_peaks, pre_samples, post_samples):
    segments = []

    for peak in r_peaks:
        start = max(0, peak - pre_samples)
        end = min(len(signal), peak + post_samples)

        # Only include segments with the correct length
        if end - start == pre_samples + post_samples:
            segment = signal[start:end]
            segments.append(segment)

    return segments

def extract_beats_multi_lead(ecg_signals, fs, pre_samples, post_samples, denoise_fn=None):
    # Convert to numpy array for consistent indexing
    ecg_signals = np.array(ecg_signals)

    # Apply denoising if provided
    if denoise_fn is not None:
        ecg_signals = np.array([denoise_fn(lead) for lead in ecg_signals])

    # Detect fused R-peaks across all leads
    r_peaks = detect_r_peaks(ecg_signals, fs)
    # print(f"Detected {len(r_peaks)} R-peaks after fusion")

    if len(r_peaks) == 0:
        return None

    # Extract segments from all leads
    all_lead_segments = []
    for lead_idx, lead_signal in enumerate(ecg_signals):
        segments = extract_segments_around_peaks(lead_signal, r_peaks, pre_samples, post_samples)
        all_lead_segments.append(segments)

    # Find the minimum number of valid segments across all leads
    min_segments = min(len(segments) for segments in all_lead_segments)
    # print(f"Extracted {min_segments} valid segments across all leads")

    if min_segments == 0:
        return None

    # Stack segments: (n_beats, segment_length, n_leads)
    beats_arr = np.stack([
        np.stack(segments[:min_segments], axis=0)
        for segments in all_lead_segments
    ], axis=-1)

    return beats_arr.astype(np.float32)

In [ ]:
import os
import h5py
import numpy as np
import pandas as pd
from collections import Counter
from joblib import Parallel, delayed
import multiprocessing as mp

# -------------------------
metadata = pd.read_csv("metadata.csv")
codes = pd.read_csv("code.csv")

code_mappings = dict(zip(codes['Code'].astype(str), codes['Category']))

metadata = metadata[~metadata['AHA_Code'].str.contains(";")]
metadata['AHA_Code'] = metadata['AHA_Code'].str.split("+").str[0]
metadata['AHA_Code_Mapped'] = metadata['AHA_Code'].map(code_mappings)

unique_labels = sorted(metadata['AHA_Code_Mapped'].dropna().unique())
label2idx = {lbl: i for i, lbl in enumerate(unique_labels)}
num_classes = len(unique_labels)

print("num_classes:", num_classes, "labels:", unique_labels)

def process_row(row):
    ecg_data_path = os.path.join(PATH, row.ECG_ID + ".h5")
    if not os.path.exists(ecg_data_path):
        return None

    label = row.AHA_Code_Mapped
    if pd.isna(label) or label not in label2idx:
        return None

    try:
        with h5py.File(ecg_data_path, 'r') as f:
            ecg = np.array(f['ecg'])
    except Exception as e:
        return None

    beats = extract_beats_multi_lead(
        ecg,
        fs=SAMPLING_RATE,
        pre_samples=PRE_PEAK_SAMPLES,
        post_samples=POST_PEAK_SAMPLES,
        denoise_fn=denoise
    )

    if beats is None:
        return None

    return (beats.astype(np.float32), label2idx[label])

# Use all physical cores by default
n_jobs = max(1, mp.cpu_count() - 1)

results = Parallel(
    n_jobs=n_jobs,
    backend="loky",
    verbose=10
)(
    delayed(process_row)(row)
    for row in metadata.itertuples(index=False)
)

data_list = [r for r in results if r is not None]

print("Prepared records:", len(data_list))
print("Beat counts distribution:", Counter([x[0].shape[0] for x in data_list]))


num_classes: 11 labels: ['A', 'C', 'D', 'E', 'F', 'H', 'I', 'J', 'K', 'L', 'M']


[Parallel(n_jobs=11)]: Using backend LokyBackend with 11 concurrent workers.
[Parallel(n_jobs=11)]: Done   3 tasks      | elapsed:    1.7s
[Parallel(n_jobs=11)]: Done  10 tasks      | elapsed:    1.8s
[Parallel(n_jobs=11)]: Done  19 tasks      | elapsed:    1.9s
[Parallel(n_jobs=11)]: Done  28 tasks      | elapsed:    2.0s
[Parallel(n_jobs=11)]: Done  39 tasks      | elapsed:    2.1s
[Parallel(n_jobs=11)]: Done  50 tasks      | elapsed:    2.2s
[Parallel(n_jobs=11)]: Batch computation too fast (0.19921389479567003s.) Setting batch_size=2.
[Parallel(n_jobs=11)]: Done  63 tasks      | elapsed:    2.3s
[Parallel(n_jobs=11)]: Done  76 tasks      | elapsed:    2.4s
[Parallel(n_jobs=11)]: Done  94 tasks      | elapsed:    2.6s
[Parallel(n_jobs=11)]: Done 124 tasks      | elapsed:    2.8s
[Parallel(n_jobs=11)]: Done 158 tasks      | elapsed:    3.0s
[Parallel(n_jobs=11)]: Done 192 tasks      | elapsed:    3.2s
[Parallel(n_jobs=11)]: Done 230 tasks      | elapsed:    3.5s
[Parallel(n_jobs=11)]

Prepared records: 22046
Beat counts distribution: Counter({11: 4756, 12: 4374, 10: 3305, 13: 3245, 14: 2113, 9: 1193, 15: 1183, 16: 483, 17: 353, 18: 250, 8: 174, 19: 135, 20: 123, 21: 76, 22: 65, 23: 32, 27: 27, 24: 27, 25: 25, 28: 18, 29: 14, 26: 14, 32: 7, 36: 6, 35: 5, 30: 5, 31: 5, 34: 4, 33: 3, 50: 2, 37: 2, 57: 2, 61: 2, 44: 2, 40: 2, 63: 2, 39: 2, 48: 2, 43: 2, 41: 2, 73: 1, 58: 1, 49: 1, 46: 1})


[Parallel(n_jobs=11)]: Done 22046 out of 22046 | elapsed:  1.5min finished


In [ ]:
print(data_list[2][0].shape), label2idx

(10, 300, 12)


(None,
 {'A': 0,
  'C': 1,
  'D': 2,
  'E': 3,
  'F': 4,
  'H': 5,
  'I': 6,
  'J': 7,
  'K': 8,
  'L': 9,
  'M': 10})

In [ ]:
# ---------- Length-bucketing Batch Sampler ----------
class LengthBucketBatchSampler(Sampler):
    def __init__(self,
                 lengths: List[int],
                 batch_size: int,
                 bin_size: Optional[int] = None,
                 shuffle: bool = True,
                 drop_last: bool = False):
        self.lengths = list(lengths)
        self.batch_size = int(batch_size)
        self.bin_size = bin_size
        self.shuffle = shuffle
        self.drop_last = drop_last

        # Build mapping length_key -> list of indices
        self._buckets = defaultdict(list)
        for idx, L in enumerate(self.lengths):
            key = self._length_key(L)
            self._buckets[key].append(idx)

        # Convert to normal dict for iteration; keep keys list stable
        self.bucket_keys = list(self._buckets.keys())

    def _length_key(self, length: int) -> int:
        if self.bin_size is None or self.bin_size <= 0:
            return int(length)   # exact-length bucket
        else:
            return (length // self.bin_size) * self.bin_size

    def __iter__(self):
      # For each epoch, build batches from buckets.
      batches = []
      for key in self.bucket_keys:
          idxs = list(self._buckets[key])
          if len(idxs) < self.batch_size:
              # skip this bucket entirely
              continue
          if self.shuffle:
              random.shuffle(idxs)
          # chunk into batches
          for i in range(0, len(idxs), self.batch_size):
              batch = idxs[i:i + self.batch_size]
              if len(batch) < self.batch_size and self.drop_last:
                  continue
              batches.append(batch)

      if self.shuffle:
          random.shuffle(batches)

      for batch in batches:
          yield batch

    def __len__(self):
      total = 0
      for key in self.bucket_keys:
          n = len(self._buckets[key])
          if n < self.batch_size:
              continue  # Skip small buckets like in __iter__
          if self.drop_last:
              total += n // self.batch_size
          else:
              total += math.ceil(n / self.batch_size)
      return total

In [ ]:
# -------------------------
# Dataset + Sampler + DataLoader
# -------------------------
class ECGSegmentDataset(Dataset):
    def __init__(self, data_list):
        # data_list: list of (beats_array (S, T, C), label_idx)
        self.data = data_list
        self.num_beats = [int(x[0].shape[0]) for x in data_list]
        # optionally check that T==SEGMENT_LENGTH for all entries
        for sig, _ in data_list:
            assert sig.shape[1] == SEGMENT_LENGTH, "SEGMENT_LENGTH mismatch"

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sig, label = self.data[idx]
        sig_t = torch.as_tensor(sig, dtype=torch.float32)  # (S, T, C)
        return {"signal": sig_t, "label": int(label), "num_beats": int(sig_t.shape[0])}


def collate_by_num_beats(batch):
    s_vals = [item["num_beats"] for item in batch]
    if not all(s == s_vals[0] for s in s_vals):
        raise ValueError("collate_by_num_beats received mixed num_beats in a batch")
    signals = torch.stack([item["signal"] for item in batch], dim=0)  # (B, S, T, C)
    labels = torch.tensor([item["label"] for item in batch], dtype=torch.long)
    return {"signal": signals, "label": labels, "num_beats": torch.tensor(s_vals, dtype=torch.long)}


In [ ]:
from torch.utils.data import Subset, DataLoader

# # instantiate dataset and sampler
ds = ECGSegmentDataset(data_list)

# -------------------------
# Split indices
# -------------------------
all_indices = list(range(len(ds)))
all_labels = [data_list[i][1] for i in all_indices]  # label_idx for each sample

# First split: train+val vs test (stratified)
trainval_indices, test_indices = train_test_split(
    all_indices,
    test_size=0.1,
    random_state=10,
    stratify=all_labels 
)

# Extract labels for the trainval subset
trainval_labels = [all_labels[i] for i in trainval_indices]

# Second split: train vs val (stratified)
train_indices, val_indices = train_test_split(
    trainval_indices,
    test_size=0.1,
    random_state=10,
    stratify=trainval_labels 
)

# -------------------------
# Create Subsets
# -------------------------
train_ds = Subset(ds, train_indices)
val_ds   = Subset(ds, val_indices)
test_ds  = Subset(ds, test_indices)

# -------------------------
# Make loader helper
# -------------------------
def make_loader(subset_ds, batch_size=16, drop_last=True, shuffle=True):
    subset_indices = subset_ds.indices
    lengths = [ds.num_beats[i] for i in subset_indices]

    sampler = LengthBucketBatchSampler(
        lengths=lengths,
        batch_size=batch_size,
        bin_size=None,       # exact-length buckets
        shuffle=shuffle,
        drop_last=drop_last
    )

    loader = DataLoader(
        subset_ds,
        batch_sampler=sampler,
        collate_fn=collate_by_num_beats,
        num_workers=6,
        pin_memory=True
    )
    return loader


In [ ]:
# Verifying stratifying
train_loader = make_loader(train_ds, batch_size=BATCH_SIZE)
val_loader   = make_loader(val_ds, batch_size=BATCH_SIZE, shuffle=False)
test_loader  = make_loader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

def get_label_distribution(indices):
    labels = [data_list[i][1] for i in indices]
    return Counter(labels)

print("Train distribution:", get_label_distribution(train_indices))
print("Val distribution:", get_label_distribution(val_indices))
print("Test distribution:", get_label_distribution(test_indices))

Train distribution: Counter({0: 11260, 9: 2198, 1: 2086, 6: 1098, 4: 409, 7: 283, 3: 215, 2: 212, 10: 45, 5: 29, 8: 21})
Val distribution: Counter({0: 1252, 9: 244, 1: 232, 6: 122, 4: 46, 7: 32, 3: 24, 2: 23, 10: 5, 5: 3, 8: 2})
Test distribution: Counter({0: 1391, 9: 271, 1: 257, 6: 135, 4: 51, 7: 35, 3: 27, 2: 26, 10: 6, 5: 3, 8: 3})


In [ ]:
# -------------------------
# Attention / helper layers
# -------------------------
class ChannelAttention(nn.Module):
    """
    Expects x shape = (batch, channels, seq_len)
    """
    def __init__(self, channels, ratio=8):
        super().__init__()
        mid = max(1, channels // ratio)
        self.mlp = nn.Sequential(
            nn.Linear(channels, mid, bias=True),
            nn.ReLU(),
            nn.Linear(mid, channels, bias=True)
        )

    def forward(self, x):
        # x: (B, C, L)
        avg_pool = torch.mean(x, dim=2)           # (B, C)
        max_pool, _ = torch.max(x, dim=2)         # (B, C)
        avg_out = self.mlp(avg_pool)              # (B, C)
        max_out = self.mlp(max_pool)              # (B, C)
        att = torch.sigmoid(avg_out + max_out)    # (B, C)
        att = att.unsqueeze(2)                    # (B, C, 1)
        return x * att                             # broadcast multiply -> (B, C, L)


class SegmentAttention(nn.Module):
    """
    Input: (batch, time_steps, input_dim)
    Produces: (batch, input_dim), (batch, time_steps)  (output vector, alphas)
    `units` is the hidden size of the attention MLP (same as original design).
    """
    def __init__(self, input_dim, units):
        super().__init__()
        self.linear = nn.Linear(input_dim, units, bias=True) 
        # u vector for scoring
        self.u = nn.Parameter(torch.randn(units))

    def forward(self, inputs):
        # inputs: (B, T, D)
        v = torch.tanh(self.linear(inputs))           # (B, T, units)
        # score each time step
        vu = torch.matmul(v, self.u)                  # (B, T)
        alphas = F.softmax(vu, dim=1)                 # softmax over time dimension
        output = torch.sum(inputs * alphas.unsqueeze(-1), dim=1)  # (B, D)
        return output, alphas


class TimeDistributedSegmentAttention(nn.Module):
    """
    inputs: (B, segments, seg_len, feature_dim)
    returns: (B, segments, feature_dim), (B, segments, seg_len)
    """
    def __init__(self, input_dim, units):
        super().__init__()
        self.segment_attention = SegmentAttention(input_dim, units)

    def forward(self, inputs):
        # inputs: (B, S, T, D)
        B, S, T, D = inputs.shape
        flat = inputs.view(B * S, T, D)                       # (B*S, T, D)
        outputs, alphas = self.segment_attention(flat)        # (B*S, D), (B*S, T)
        outputs = outputs.view(B, S, D)                       # (B, S, D)
        alphas = alphas.view(B, S, T)                         # (B, S, T)
        return outputs, alphas


class HANWithAttention(nn.Module):
    def __init__(
        self,
        num_classes,
        conv_channels=128,
        segment_hidden=256,
        sequence_hidden=512,
        fc_hidden=2048,
        dropout=0.4,
    ):
        super().__init__()

        # 1D conv over 12 leads
        self.conv1d = nn.Conv1d(12, conv_channels, kernel_size=25, padding=12)
        self.channel_attention = ChannelAttention(conv_channels, ratio=8)
        self.pool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)

        # Beat-level LSTM
        self.lstm_segment = nn.LSTM(
            input_size=conv_channels,
            hidden_size=segment_hidden,
            batch_first=True
        )
        self.time_distributed_attention = TimeDistributedSegmentAttention(
            segment_hidden,
            segment_hidden
        )

        # Sequence-level LSTM
        self.lstm_sequence = nn.LSTM(
            input_size=segment_hidden,
            hidden_size=sequence_hidden,
            batch_first=True
        )
        self.final_attention = SegmentAttention(sequence_hidden, sequence_hidden)

        self.fc = nn.Linear(sequence_hidden, fc_hidden)
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(fc_hidden, num_classes)

    def forward(self, x):
        logits, _, _ = self.forward_with_attention(x)
        return logits

    def forward_with_attention(self, x):
        # x: (B, S, T, C) = (batch, beats, time, leads)
        B, S, T, C = x.shape

        # process beats individually: (B*S, C, T)
        x = x.view(B * S, T, C).permute(0, 2, 1)

        conv = self.conv1d(x)
        att  = self.channel_attention(conv)
        pooled = self.pool(att).permute(0, 2, 1)     # (B*S, T2, conv_channels)

        seg_out, _ = self.lstm_segment(pooled)       # (B*S, T2, segment_hidden)

        # regroup beats: (B, S, T2, segment_hidden)
        seg_out = seg_out.view(B, S, seg_out.shape[1], seg_out.shape[2])

        # TimeDistributed attention over each beat
        seg_feats, seg_alphas = self.time_distributed_attention(seg_out)  # (B,S,segment_hidden)

        # Sequence-level LSTM over beats
        seq_out, _ = self.lstm_sequence(seg_feats)                        # (B,S,sequence_hidden)
        final_vec, final_alphas = self.final_attention(seq_out)           # (B,sequence_hidden), (B,S)

        x = F.relu(self.fc(final_vec))
        x = self.dropout(x)
        logits = self.classifier(x)                                       # (B,C)
        return logits, final_alphas, seg_alphas

In [ ]:
from torchinfo import summary

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HANWithAttention(num_classes=11,
                          conv_channels=128,
                          segment_hidden=128,
                          sequence_hidden=512,
                          fc_hidden=512,
                          dropout=0.3).to(device)

summary(model,
        input_size=(2, 10, 300, 12),    # (batch, segments, timesteps, channels)
        col_names=("input_size", "output_size", "num_params", "trainable"),
        depth=4,
        device=device.type)            # "cpu" or "cuda"


Layer (type:depth-idx)                        Input Shape               Output Shape              Param #                   Trainable
HANWithAttention                              [2, 10, 300, 12]          [2, 11]                   --                        True
├─Conv1d: 1-1                                 [20, 12, 300]             [20, 128, 300]            38,528                    True
├─ChannelAttention: 1-2                       [20, 128, 300]            [20, 128, 300]            --                        True
│    └─Sequential: 2-1                        [20, 128]                 [20, 128]                 --                        True
│    │    └─Linear: 3-1                       [20, 128]                 [20, 16]                  2,064                     True
│    │    └─ReLU: 3-2                         [20, 16]                  [20, 16]                  --                        --
│    │    └─Linear: 3-3                       [20, 16]                  [20, 128]             

In [ ]:
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

from tqdm import tqdm

def evaluate_metrics(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_labels, all_preds, all_probs = [], [], []

    with torch.no_grad():
        for batch in loader:
            inputs = batch["signal"].to(device)
            labels = batch["label"].to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)

            probs = torch.softmax(outputs, dim=1).cpu().numpy()
            preds = outputs.argmax(dim=1).cpu().numpy()
            labels = labels.cpu().numpy()

            all_labels.extend(labels)
            all_preds.extend(preds)
            all_probs.extend(probs)

    avg_loss = running_loss / len(all_labels)
    acc = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average="weighted", zero_division=0)
    recall = recall_score(all_labels, all_preds, average="weighted", zero_division=0)
    f1 = f1_score(all_labels, all_preds, average="weighted", zero_division=0)

    try:
      y_true = np.eye(num_classes)[all_labels]
      y_score = np.array(all_probs)

      auc_list = []
      for i in range(num_classes):
          if np.any(y_true[:, i]):  # class i exists
              auc_list.append(roc_auc_score(y_true[:, i], y_score[:, i]))
      if auc_list:
          auc = np.mean(auc_list)
      else:
          auc = float("nan")
    except ValueError:
        auc = float("nan")

    return avg_loss, acc, precision, recall, auc, f1

def train(model, train_loader, val_loader, optimizer, criterion, device,
          epochs, scheduler=None):
    model.to(device)
    history = {
        "train_loss": [], "train_acc": [], "train_f1": [], "train_precision": [], "train_recall": [], "train_auc": [],
        "val_loss": [],   "val_acc": [],   "val_f1": [],   "val_precision": [],   "val_recall": [], "val_auc": []
    }

    for epoch in range(1, epochs + 1):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        all_preds = []
        all_probs = []
        all_labels = []
        steps = 0

        loop = tqdm(train_loader, desc=f"Epoch {epoch}/{epochs}", leave=False)
        for batch in loop:
            inputs = batch["signal"].to(device)
            labels = batch["label"].to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            batch_size = labels.size(0)
            running_loss += loss.item() * batch_size
            preds = outputs.argmax(dim=1)
            probs = torch.softmax(outputs, dim=1).detach().cpu().numpy()
            correct += (preds == labels).sum().item()
            total += batch_size

            all_preds.extend(preds.cpu().numpy().tolist())
            all_probs.extend(probs.tolist())
            all_labels.extend(labels.cpu().numpy().tolist())

            steps += 1
            loop.set_postfix(loss=f"{loss.item():.4f}", acc=f"{(correct/total):.4f}")

        if scheduler is not None:
            try:
                scheduler.step()
            except Exception:
                pass

        if total == 0:
            epoch_loss = float("nan")
            epoch_acc = float("nan")
            epoch_precision = epoch_recall = epoch_f1 = float("nan")
        else:
            epoch_loss = running_loss / total
            epoch_acc = correct / total
            epoch_precision = precision_score(all_labels, all_preds, average="weighted", zero_division=0)
            epoch_recall = recall_score(all_labels, all_preds, average="weighted", zero_division=0)
            epoch_f1 = f1_score(all_labels, all_preds, average="weighted", zero_division=0)
            try:
              y_true = np.eye(num_classes)[all_labels]
              y_score = np.array(all_probs)

              auc_list = []
              for i in range(num_classes):
                  if np.any(y_true[:, i]):
                      auc_list.append(roc_auc_score(y_true[:, i], y_score[:, i]))
              if auc_list:
                  auc = np.mean(auc_list)
              else:
                  auc = float("nan")
            except ValueError:
                auc = float("nan")

        # append train history
        history["train_loss"].append(epoch_loss)
        history["train_auc"].append(auc)
        history["train_acc"].append(epoch_acc)
        history["train_precision"].append(epoch_precision)
        history["train_recall"].append(epoch_recall)
        history["train_f1"].append(epoch_f1)

        print(f"Epoch {epoch}/{epochs}  train_loss: {epoch_loss:.4f}  train_acc: {epoch_acc:.4f}  train_f1: {epoch_f1:.4f}")

        # Validation
        if val_loader is not None:
            vloss, vacc, vprecision, vrecall, vauc, vf1 = evaluate_metrics(model, val_loader, criterion, device)
            history["val_loss"].append(vloss)
            history["val_acc"].append(vacc)
            history["val_auc"].append(vauc)
            history["val_f1"].append(vf1)
            history["val_precision"].append(vprecision)
            history["val_recall"].append(vrecall)


            if 'mlflow' in globals():
                try:
                    mlflow.log_metric("train_loss", epoch_loss, step=epoch)
                    mlflow.log_metric("train_f1", epoch_f1, step=epoch)
                    mlflow.log_metric("train_precision", epoch_precision, step=epoch)
                    mlflow.log_metric("train_recall", epoch_recall, step=epoch)
                    mlflow.log_metric("train_acc", epoch_acc, step=epoch)
                    mlflow.log_metric("train_auc", auc, step=epoch)

                    mlflow.log_metric("val_loss", vloss, step=epoch)
                    mlflow.log_metric("val_f1", vf1, step=epoch)
                    mlflow.log_metric("val_precision", vprecision, step=epoch)
                    mlflow.log_metric("val_recall", vrecall, step=epoch)
                    mlflow.log_metric("val_acc", vacc, step=epoch)
                    mlflow.log_metric("val_auc", vauc, step=epoch)
                except Exception:
                    pass

            tqdm.write(
                f"Epoch {epoch}  train_loss {epoch_loss:.4f} | val_loss {vloss:.4f} | "
                f"val_f1 {vf1:.4f} | val_precision {vprecision:.4f} | val_recall {vrecall:.4f} | val_auc {vauc:.4f}"
            )
        else:
            # keep consistent history lengths
            history["val_loss"].append(None)
            history["val_auc"].append(None)
            history["val_acc"].append(None)
            history["val_f1"].append(None)
            history["val_precision"].append(None)
            history["val_recall"].append(None)

    return history

# -------------


In [ ]:


import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from tqdm import tqdm
import random

def run_experiments(model_fn,
                    optimizer_fn, criterion, device, epochs, num_classes,
                    seeds=10):

  results = {"acc": [], "precision": [], "recall": [], "auc": [], "loss": []}
  print("run exp")
  for seed in range(seeds):
      print(f"\n=== Seed {seed} ===")
      set_seed(seed)

      all_indices = list(range(len(ds)))
      all_labels = [data_list[i][1] for i in all_indices]  # label_idx for each sample

      # First split: train+val vs test (stratified)
      trainval_indices, test_indices = train_test_split(
          all_indices,
          test_size=0.1,
          random_state=seed,
          stratify=all_labels  # ← stratify by class labels
      )

      # Extract labels for the trainval subset
      trainval_labels = [all_labels[i] for i in trainval_indices]

      # Second split: train vs val (stratified)
      train_indices, val_indices = train_test_split(
          trainval_indices,
          test_size=0.1,
          random_state=seed,
          stratify=trainval_labels  # ← stratify by class labels
      )

      # -------------------------
      # Create Subsets
      # -------------------------
      train_ds = Subset(ds, train_indices)
      val_ds   = Subset(ds, val_indices)
      test_ds  = Subset(ds, test_indices)

      train_loader = make_loader(train_ds, batch_size=BATCH_SIZE)
      val_loader   = make_loader(val_ds, batch_size=BATCH_SIZE, shuffle=False)
      test_loader  = make_loader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

      print("Train distribution:", get_label_distribution(train_indices))
      print("Val distribution:", get_label_distribution(val_indices))
      print("Test distribution:", get_label_distribution(test_indices))

      model = model_fn().to(device)
      optimizer = optimizer_fn(model)
      scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

      # Train
      with mlflow.start_run(nested=True, run_name=f"HAN-SPH-SEED-{seed}"):
        _ = train(model, train_loader, val_loader, optimizer, criterion, device, epochs, scheduler)

        # Evaluate on test set
        loss, acc, precision, recall, auc, f1 = evaluate_metrics(model, test_loader, criterion, device)

        print(f"Test (seed {seed}) — loss: {loss:.4f}, acc: {acc:.4f}, "
            f"precision: {precision:.4f}, recall: {recall:.4f}, auc: {auc:.4f}, f1: {f1:.4f}")

      results["loss"].append(loss)
      results["acc"].append(acc)
      results["precision"].append(precision)
      results["recall"].append(recall)
      results["auc"].append(auc)

  # Aggregate results
  print("\n=== Final Results (across seeds) ===")
  for k, v in results.items():
      arr = np.array(v, dtype=np.float32)
      print(f"{k}: mean={arr.mean():.4f}, std={arr.std():.4f}")

  return results


In [ ]:
def model_fn():
  return HANWithAttention(num_classes=num_classes,
                            conv_channels=128,
                            segment_hidden=128,
                            sequence_hidden=512,
                            fc_hidden=512,
                            dropout=0.3)

def optimizer_fn(model):
  return torch.optim.Adam(model.parameters(), lr=0.003, weight_decay=9e-5)


In [ ]:
criterion = nn.CrossEntropyLoss()
run_experiments(model_fn, optimizer_fn, criterion, DEVICE, 25, 11, 10)
# maybe change num of epochs to 30

run exp

=== Seed 0 ===
Train distribution: Counter({0: 11260, 9: 2198, 1: 2086, 6: 1098, 4: 409, 7: 283, 3: 215, 2: 212, 10: 45, 5: 29, 8: 21})
Val distribution: Counter({0: 1252, 9: 244, 1: 232, 6: 122, 4: 46, 7: 32, 3: 24, 2: 23, 10: 5, 5: 3, 8: 2})
Test distribution: Counter({0: 1391, 9: 271, 1: 257, 6: 135, 4: 51, 7: 35, 3: 27, 2: 26, 10: 6, 8: 3, 5: 3})


Epoch 1/25  train_loss: 1.2474  train_acc: 0.6445  train_f1: 0.5288
Epoch 1  train_loss 1.2474 | val_loss 1.0359 | val_f1 0.6396 | val_precision 0.6517 | val_recall 0.7235 | val_auc 0.6354


Epoch 2/25  train_loss: 1.0432  train_acc: 0.7020  train_f1: 0.6222
Epoch 2  train_loss 1.0432 | val_loss 0.9021 | val_f1 0.6835 | val_precision 0.7087 | val_recall 0.7434 | val_auc 0.7577


Epoch 3/25  train_loss: 0.9198  train_acc: 0.7312  train_f1: 0.6672
Epoch 3  train_loss 0.9198 | val_loss 0.8363 | val_f1 0.6937 | val_precision 0.7346 | val_recall 0.7489 | val_auc 0.7728


Epoch 4/25  train_loss: 0.8817  train_acc: 0.7385  train_f1: 0.6774
Epoch 4  train_loss 0.8817 | val_loss 0.8077 | val_f1 0.6936 | val_precision 0.7523 | val_recall 0.7561 | val_auc 0.8005


...

Epoch 22/25  train_loss: 0.3777  train_acc: 0.8668  train_f1: 0.8592
Epoch 22  train_loss 0.3777 | val_loss 0.5817 | val_f1 0.7989 | val_precision 0.8014 | val_recall 0.8141 | val_auc 0.9075


Epoch 23/25  train_loss: 0.3696  train_acc: 0.8697  train_f1: 0.8631
Epoch 23  train_loss 0.3696 | val_loss 0.5899 | val_f1 0.7992 | val_precision 0.8042 | val_recall 0.8157 | val_auc 0.9107


Epoch 24/25  train_loss: 0.3583  train_acc: 0.8753  train_f1: 0.8690
Epoch 24  train_loss 0.3583 | val_loss 0.6100 | val_f1 0.7952 | val_precision 0.7997 | val_recall 0.8109 | val_auc 0.9100


Epoch 25/25  train_loss: 0.3450  train_acc: 0.8821  train_f1: 0.8762
Epoch 25  train_loss 0.3450 | val_loss 0.6019 | val_f1 0.8038 | val_precision 0.8007 | val_recall 0.8157 | val_auc 0.9081
Test (seed 9) — loss: 0.6665, acc: 0.7974, precision: 0.7883, recall: 0.7974, auc: 0.9162, f1: 0.7896
🏃 View run HAN-SPH-SEED-9 at: https://dbc-5cba9d5d-ddf8.cloud.databricks.com/ml/experiments/4416535780169939/runs/75ffb9d85f9642a58e4435c04082101f
🧪 View experiment at: https://dbc-5cba9d5d-ddf8.cloud.databricks.com/ml/experiments/4416535780169939

=== Final Results (across seeds) ===
acc: mean=0.8008, std=0.0065
precision: mean=0.7886, std=0.0080
recall: mean=0.8008, std=0.0065
auc: mean=0.8973, std=0.0252
loss: mean=0.6350, std=0.0263


{'acc': [0.7951923076923076,
  0.7926136363636364,
  0.799618320610687,
  0.811531007751938,
  0.8076923076923077,
  0.8057170542635659,
  0.8067748091603053,
  0.7999031007751938,
  0.7916666666666666,
  0.79736328125],
 'precision': [0.7800924986177286,
  0.7797396395410207,
  0.7858092434940311,
  0.7982302919347486,
  0.7963105054100662,
  0.7939990964623084,
  0.8001345735415405,
  0.7877252577161785,
  0.7755182952021622,
  0.7883371137884427],
 'recall': [0.7951923076923076,
  0.7926136363636364,
  0.799618320610687,
  0.811531007751938,
  0.8076923076923077,
  0.8057170542635659,
  0.8067748091603053,
  0.7999031007751938,
  0.7916666666666666,
  0.79736328125],
 'auc': [np.float64(0.8953378055180958),
  np.float64(0.8943828418789028),
  np.float64(0.8756571148852204),
  np.float64(0.864071943595428),
  np.float64(0.8571767554742742),
  np.float64(0.9336728634700737),
  np.float64(0.9325294393238616),
  np.float64(0.9112268256903633),
  np.float64(0.8931580878559662),
  np.floa

In [ ]:
summary(model,
        input_size=(2, 10, 300, 12),    # (batch, segments, timesteps, channels)
        col_names=("input_size", "output_size", "num_params", "trainable"),
        depth=4,
        device=device.type)

Layer (type:depth-idx)                        Input Shape               Output Shape              Param #                   Trainable
HANWithAttention                              [2, 10, 300, 12]          [2, 11]                   --                        True
├─Conv1d: 1-1                                 [20, 12, 300]             [20, 128, 300]            38,528                    True
├─ChannelAttention: 1-2                       [20, 128, 300]            [20, 128, 300]            --                        True
│    └─Sequential: 2-1                        [20, 128]                 [20, 128]                 --                        True
│    │    └─Linear: 3-1                       [20, 128]                 [20, 16]                  2,064                     True
│    │    └─ReLU: 3-2                         [20, 16]                  [20, 16]                  --                        --
│    │    └─Linear: 3-3                       [20, 16]                  [20, 128]             

# Resource use

In [ ]:
!pip install calflops -q

In [ ]:
import calflops

In [ ]:
model = HANWithAttention(num_classes=11,
                            conv_channels=128,
                            segment_hidden=128,
                            sequence_hidden=512,
                            fc_hidden=512,
                            dropout=0.3)

flops, macs, params = calflops.calculate_flops(model=model,
                                      input_shape=(1, 10, 300, 12),
                                      output_as_string=True,
                                      output_precision=4)
print("Model FLOPs:%s   MACs:%s   Params:%s \n" %(flops, macs, params))


------------------------------------- Calculate Flops Results -------------------------------------
Notations:
number of parameters (Params), number of multiply-accumulate operations(MACs),
number of floating-point operations (FLOPs), floating-point operations per second (FLOPS),
fwd FLOPs (model forward propagation FLOPs), bwd FLOPs (model backward propagation FLOPs),
default model backpropagation takes 2.00 times as much computation as forward propagation.

Total Training Params:                                                  2.04 M  
fwd MACs:                                                               169.945 MMACs
fwd FLOPs:                                                              762.061 MFLOPS
fwd+bwd MACs:                                                           509.834 MMACs
fwd+bwd FLOPs:                                                          2.2862 GFLOPS

-------------------------------- Detailed Calculated FLOPs Results --------------------------------
Each mod

In [ ]:
def analyze_model_complexity(model, input_size=(1, 10, 300, 12), device='cuda'):
    """
    Analyze model complexity: parameters, FLOPs, memory

    Args:
        model: Your HANWithAttention model
        input_size: (batch, segments, timesteps, channels)
        device: 'cuda' or 'cpu'
    """
    print("="*80)
    print("MODEL COMPLEXITY ANALYSIS")
    print("="*80)

    model = model.to(device)
    model.eval()

    # 1. Parameter count
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    non_trainable_params = total_params - trainable_params

    print(f"\n📊 Parameter Count:")
    print(f"   Total Parameters:       {total_params:,}")
    print(f"   Trainable Parameters:   {trainable_params:,}")
    print(f"   Non-trainable Parameters: {non_trainable_params:,}")
    print(f"   Model Size (MB):        {total_params * 4 / (1024**2):.2f}")  # 4 bytes per float32

    # 2. FLOPs calculation
    dummy_input = torch.randn(input_size).to(device)

    try:
        macs, params = profile(model, inputs=(dummy_input,), verbose=False)
        macs, params = clever_format([macs, params], "%.3f")
        print(f"\n🔢 FLOPs (Floating Point Operations):")
        print(f"   MACs (Multiply-Accumulate): {macs}")
        print(f"   Parameters (thop):          {params}")
    except Exception as e:
        print(f"\n⚠️  FLOPs calculation failed: {e}")

    # 3. Layer-wise parameter breakdown
    print(f"\n📋 Layer-wise Parameter Breakdown:")
    print(f"{'Layer Name':<40} {'Parameters':>15} {'% of Total':>12}")
    print("-"*70)

    for name, param in model.named_parameters():
        if param.requires_grad:
            param_count = param.numel()
            percentage = 100 * param_count / trainable_params
            print(f"{name:<40} {param_count:>15,} {percentage:>11.2f}%")

    # 4. Detailed model summary
    print(f"\n📝 Detailed Model Architecture:")
    summary(model,
            input_size=input_size,
            col_names=["input_size", "output_size", "num_params", "mult_adds"],
            depth=4,
            device=device,
            verbose=0)

    return {
        'total_params': total_params,
        'trainable_params': trainable_params,
        'model_size_mb': total_params * 4 / (1024**2)
    }

analyze_model_complexity(model)

MODEL COMPLEXITY ANALYSIS

📊 Parameter Count:
   Total Parameters:       2,037,787
   Trainable Parameters:   2,037,787
   Non-trainable Parameters: 0
   Model Size (MB):        7.77

🔢 FLOPs (Floating Point Operations):
   MACs (Multiply-Accumulate): 355.616M
   Parameters (thop):          2.037M

📋 Layer-wise Parameter Breakdown:
Layer Name                                    Parameters   % of Total
----------------------------------------------------------------------
conv1d.weight                                     38,400        1.88%
conv1d.bias                                          128        0.01%
channel_attention.mlp.0.weight                     2,048        0.10%
channel_attention.mlp.0.bias                          16        0.00%
channel_attention.mlp.2.weight                     2,048        0.10%
channel_attention.mlp.2.bias                         128        0.01%
lstm_segment.weight_ih_l0                         65,536        3.22%
lstm_segment.weight_hh_l0          

{'total_params': 2037787,
 'trainable_params': 2037787,
 'model_size_mb': 7.773540496826172}

In [ ]:
# From: https://discuss.pytorch.org/t/finding-model-size/130275

param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('model size: {:.3f}MB'.format(size_all_mb))


model size: 7.774MB


In [ ]:
import time
import numpy as np
import torch

# =============================================================================
# Benchmark Environment Setup
# =============================================================================

def setup_benchmark_env():
    """
    Freeze backend behavior for reproducible benchmarking.
    """
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    torch.backends.cuda.matmul.allow_tf32 = False
    torch.backends.cudnn.allow_tf32 = False

def time_forward(model, inputs, device='cuda'):
    """
    Time a single forward pass with proper synchronization.
    Returns elapsed time in seconds.
    """
    if device == 'cuda':
        torch.cuda.synchronize()
        start = torch.cuda.Event(enable_timing=True)
        end = torch.cuda.Event(enable_timing=True)

        start.record()
        _ = model(inputs)
        end.record()

        torch.cuda.synchronize()
        return start.elapsed_time(end) / 1000.0  # seconds
    else:
        t0 = time.time()
        _ = model(inputs)
        return time.time() - t0


# =============================================================================
# Batch-1 Latency Benchmark
# =============================================================================

def benchmark_latency(
    model,
    input_shape,
    device='cuda',
    runs=100,
    warmup=20
):
    """
    Measure true per-sample inference latency (batch size = 1).
    """
    model = model.to(device).eval()

    with torch.no_grad():
        # Warmup
        for _ in range(warmup):
            x = torch.randn(input_shape, device=device)
            _ = model(x)
            if device == 'cuda':
                torch.cuda.synchronize()

        times = []
        for _ in range(runs):
            x = torch.randn(input_shape, device=device)
            elapsed = time_forward(model, x, device)
            times.append(elapsed)

    times = np.array(times)

    return {
        "mean_ms": times.mean() * 1000,
        "std_ms": times.std() * 1000,
        "median_ms": np.median(times) * 1000,
        "min_ms": times.min() * 1000,
        "max_ms": times.max() * 1000,
    }


# =============================================================================
# Batch-N Throughput Benchmark
# =============================================================================

def benchmark_throughput(
    model,
    input_shape,
    batch_size,
    device='cuda',
    runs=100,
    warmup=20
):
    """
    Measure throughput under batched inference.
    """
    model = model.to(device).eval()
    shape = (batch_size,) + input_shape[1:]

    with torch.no_grad():
        # Warmup
        for _ in range(warmup):
            x = torch.randn(shape, device=device)
            _ = model(x)
            if device == 'cuda':
                torch.cuda.synchronize()

        times = []
        for _ in range(runs):
            x = torch.randn(shape, device=device)
            elapsed = time_forward(model, x, device)
            times.append(elapsed)

    times = np.array(times)

    return {
        "batch_time_ms": times.mean() * 1000,
        "per_sample_ms": (times.mean() / batch_size) * 1000,
        "samples_per_sec": batch_size / times.mean()
    }

def run_inference_benchmark(
    model,
    input_shape=(1, 10, 300, 12),
    batch_size=16,
    device='cuda'
):
    setup_benchmark_env()

    print("=" * 80)
    print("INFERENCE BENCHMARK")
    print("=" * 80)
    print(f"Device: {device}")
    print(f"Input shape: {input_shape}")
    print(f"Throughput batch size: {batch_size}")

    latency = benchmark_latency(
        model=model,
        input_shape=input_shape,
        device=device
    )

    throughput = benchmark_throughput(
        model=model,
        input_shape=input_shape,
        batch_size=batch_size,
        device=device
    )

    print("\nBatch-1 Latency (ms):")
    for k, v in latency.items():
        print(f"  {k:>10}: {v:.3f}")

    print("\nBatch-N Throughput:")
    for k, v in throughput.items():
        if "ms" in k:
            print(f"  {k:>16}: {v:.3f}")
        else:
            print(f"  {k:>16}: {v:.2f}")

    return {
        "latency": latency,
        "throughput": throughput
    }

results = run_inference_benchmark(
    model,
    input_shape=(1, 10, 300, 12),
    batch_size=16,
    device='cuda'
)


INFERENCE BENCHMARK
Device: cuda
Input shape: (1, 10, 300, 12)
Throughput batch size: 16

Batch-1 Latency (ms):
     mean_ms: 1.716
      std_ms: 0.099
   median_ms: 1.693
      min_ms: 1.629
      max_ms: 2.359

Batch-N Throughput:
     batch_time_ms: 3.276
     per_sample_ms: 0.205
   samples_per_sec: 4883.44
